In [ ]:
# Convert groundtruth plys to groundtruth label txts

from src.evaluation_and_conversion import painted_ply_to_label_dict, label_dict_to_label_txt

import os

folder = "../../Data/fumane/groundtruths/"

broken = []
for file in os.listdir(folder+"plys/"):
    print("Convert file ", file)
    file_id = file.split("_c.ply")[0]
    try:
        gt_label_dict = painted_ply_to_label_dict(folder+"plys/"+file, clean_thresh=10, connected_components = True)
        label_dict_to_label_txt(gt_label_dict, folder+"txts_test4/"+file_id+"_gt_labels")
    except:
        broken.append(file_id)

print("Broken files:")
print(broken)

In [5]:
# evaluate

from src.evaluation_and_conversion import painted_ply_to_label_dict, compare_result_txt_to_groundtruth_label_dict, label_txt_to_label_dict

import os
import csv

folder = "../../Data/fumane_test/"

with open(folder + "/results/best_results.txt", "w") as fresults:
    fresults.write("ObjectID BestResult BestParams\n")
    
    for dir_name in os.listdir(folder+"results/"):
        obj_id = dir_name.split("_GMOCF_r1.00_n4_v256.volume", 1)[0]
        gt_filepath = folder+"groundtruths/txts/" + obj_id + "_gt_labels.txt"
        result_files = []
        
        if dir_name != "best_results.txt":
            for filename in os.listdir(folder+"results/"+dir_name+"/"):

                if filename.endswith(".txt") and not filename.endswith("_correct.txt") and not filename.endswith("_timings.txt"):
                    result_files.append(os.path.join(folder+"results/"+dir_name+"/", filename))
                else:
                    continue

            gt_label = label_txt_to_label_dict(gt_filepath)

            best_result = 0
            best_param = None

            result_table = []
            #result_table.append(["Persistence", "High_thresh", "Low_thresh", "Merge_thresh", "Correctness"])
            for file in result_files:
                correctness, high_thr, low_thr, merge_thr = compare_result_txt_to_groundtruth_label_dict(file, gt_label, metric="IoU", plot_correctness_mask=False)
                result_table.append([high_thr, low_thr, merge_thr, correctness])
                if correctness > best_result:
                    best_result = correctness
                    best_param = [high_thr, low_thr, merge_thr]

            fresults.write(str(obj_id) + " " + str(best_result) + " " + str(best_param) + "\n")

            with open(folder+"results/"+dir_name+"/"+str(obj_id)+"_result_table.csv", "w", newline="") as f:
                writer = csv.writer(f)
                writer.writerows(result_table)

label_txt_to_label_dict took 0.08566 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 15.75236 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 21.46510 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 21.32178 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 22.74544 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 21.23418 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 22.71099 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 21.11781 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 21.34946 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 15.85507 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 21.17196 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 22.44231 seconds to execute!
compare_result_txt_to_groundtruth_label_dict took 22.64548 seconds to exec